In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import pickle
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable

desired_frames = 1
desired_features = 1458

In [2]:
labels = pd.read_csv("scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
file = open('audio_embed_final.pickle', 'rb')
audio_data = pickle.load(file)
file.close()

In [7]:
file = open('text_embed_pca_final.pickle', 'rb')
text_data = pickle.load(file)
file.close()

In [8]:
at_data = {}
for scene in scenes:
    at_data[scene] = np.concatenate((audio_data[scene], text_data[scene]), axis=1)


In [9]:
at_data['1_10004'].shape

(18, 1458)

In [10]:
def get_model_data(at_features):
    model_data = pd.DataFrame(columns=['at_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
        model_data = model_data.append({'at_feature': at_features[row["SCENE"]],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [11]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data
   

In [12]:
class RNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_at_feature']
            a=np.empty((18,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_at_feature']
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)
    
class RNNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
class RNNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
def evaluateRNN(rnn, review, size):
    output = rnn(review)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(rnn, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateRNN(rnn, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals

In [13]:
warnings.filterwarnings("ignore")

model_data = get_model_data(at_data)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,at_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[-637.1869506835938, 10.25528335571289, -3.98...",0.0,NONE,SHELDON,25
1,"[[-625.8624267578125, 51.68547058105469, 40.30...",0.0,NONE,PENNY,15
2,"[[-500.3988952636719, 21.715717315673828, 18.9...",0.0,NONE,RAJ,21
3,"[[-313.1777038574219, 97.45339965820312, -58.0...",1.0,PRO,HOWARD,7
4,"[[-337.88116455078125, 107.24081420898438, -49...",0.0,NONE,SHELDON,25


In [14]:
train_data, test_data = get_train_test_split(model_data, ['at_feature', 'speaker_encode'], 'sarcasm', 'sarcasm_type')
rnn_train = train_data.copy()
rnn_test = test_data.copy()
rnn_train.reset_index(drop=True, inplace = True)
rnn_test.reset_index(drop=True, inplace = True)

Train:  (961, 2) (961,) Test:  ((241, 2), (241,))


In [15]:
desired_length = 18

rnn_train['padded_at_feature'] = rnn_train.loc[:, 'at_feature']
rnn_test['padded_at_feature'] = rnn_test.loc[:, 'at_feature']

rnn_train["sarcasm"] = rnn_train["sarcasm"].astype('int').to_numpy()
rnn_test["sarcasm"] = rnn_test["sarcasm"].astype('int').to_numpy()

In [16]:
EMBEDDING_DIM_sid = 1458
EMBEDDING_DIM_sd = 1459
HIDDEN_DIM = 20
OUTPUT_DIM = 2
layers = 2
criterion = nn.NLLLoss()

### Speaker Dependent

In [17]:
rnn_train_tensor = RNNTensorDataset(rnn_train[['padded_at_feature', 'speaker_encode','sarcasm']], True)
rnn_test_tensor = RNNTensorDataset(rnn_test[['padded_at_feature', 'speaker_encode', 'sarcasm']], True)

num_of_workers = 0
batch_size = 44
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [18]:
rnn = RNNetSD(EMBEDDING_DIM_sd, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)


RNNetSD(
  (rnn): RNN(1459, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)


In [19]:
n_epochs = 101
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%10 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5021    0.9833    0.6648       120
           1     0.6667    0.0331    0.0630       121

    accuracy                         0.5062       241
   macro avg     0.5844    0.5082    0.3639       241
weighted avg     0.5847    0.5062    0.3626       241

Epoch: 10
              precision    recall  f1-score   support

           0     0.6131    0.7000    0.6537       120
           1     0.6538    0.5620    0.6044       121

    accuracy                         0.6307       241
   macro avg     0.6335    0.6310    0.6291       241
weighted avg     0.6336    0.6307    0.6290       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.5800    0.7250    0.6444       120
           1     0.6374    0.4793    0.5472       121

    accuracy                         0.6017       241
   macro avg     0.6087    0.6022    0.5958       241
weighted avg     0.6088    0.6017    0.5956  